In [63]:
import tensorflow as tf
import os
import time
import numpy as np
import glob
from PIL import Image

import grpc
from tensorflow_serving.apis import predict_pb2
from tensorflow_serving.apis import prediction_service_pb2_grpc

## Download model ##

In [33]:
def download_model():
    base_url = 'http://download.tensorflow.org/models/object_detection/faster_rcnn_resnet101_coco_2018_01_28.tar.gz'
    !mkdir model_weights
    !wget {base_url} && mv faster_rcnn_resnet101_coco_2018_01_28.tar.gz model_weights
    !tar -xvf model_weights/faster_rcnn_resnet101_coco_2018_01_28.tar.gz -C model_weights
    !cd model_weights/faster_rcnn_resnet101_coco_2018_01_28/saved_model && mkdir 1 && mv variables 1 && mv saved_model.pb 1

In [34]:
download_model()

In [93]:
def serve_model():
    import docker
    client = docker.from_env()

    for container in client.containers.list(all=True):
        if container.name == 'tf-serving':
            container.remove(force=True)
            
    model_dir = os.path.abspath('./') + os.sep + 'model_weights/faster_rcnn_resnet101_coco_2018_01_28/saved_model_manual/saved_model/'
    # Serve the detector and the classifiers
    volume_bindings = {
        model_dir: {
            'bind': '/models/detector/',
            'mode': 'rw'
        }
    }

    container = client.containers.run('tensorflow/serving:2.3.0-gpu', 
                                      remove=True,
                                      name='tf-serving',
                                      runtime='nvidia', 
                                      command='--model_base_path=/models/detector --model_name=detector',
                                      volumes=volume_bindings,
                                      ports={'8500/tcp': '8510'},
                                      detach=True)
    time.sleep(3)
    print(container.logs(stdout=True, stderr=True).decode('utf-8'))

    return container

In [100]:
serve_model()

2020-09-11 21:24:39.798416: I tensorflow_serving/model_servers/server.cc:87] Building single TensorFlow model file config:  model_name: detector model_base_path: /models/detector
2020-09-11 21:24:39.798520: I tensorflow_serving/model_servers/server_core.cc:464] Adding/updating models.
2020-09-11 21:24:39.798528: I tensorflow_serving/model_servers/server_core.cc:575]  (Re-)adding model: detector
2020-09-11 21:24:39.898750: I tensorflow_serving/core/basic_manager.cc:739] Successfully reserved resources to load servable {name: detector version: 1}
2020-09-11 21:24:39.898761: I tensorflow_serving/core/loader_harness.cc:66] Approving load for servable version {name: detector version: 1}
2020-09-11 21:24:39.898765: I tensorflow_serving/core/loader_harness.cc:74] Loading servable version {name: detector version: 1}
2020-09-11 21:24:39.898778: I external/org_tensorflow/tensorflow/cc/saved_model/reader.cc:31] Reading SavedModel from: /models/detector/1
2020-09-11 21:24:39.979545: I external/org

<Container: e34a9b894d>

In [102]:
imgs_array = np.array([np.array(Image.open(img_file).resize((1920, 1080))) for img_file in glob.glob("./sample-images/*.jpg")])
print("IMGS ARRAY SHAPE")
print(imgs_array.shape)

# tf-serving communication setup
options = [('grpc.max_receive_message_length', 100 * 4000 * 4000)]
channel = grpc.insecure_channel('{host}:{port}'.format(host='localhost', port='8510'), options = options)
stub = prediction_service_pb2_grpc.PredictionServiceStub(channel)
request = predict_pb2.PredictRequest()
request.model_spec.name = 'detector'
request.model_spec.signature_name = 'serving_default'

t0 = time.time()

request.inputs['inputs'].CopyFrom(tf.make_tensor_proto(imgs_array, shape=imgs_array.shape))
res = stub.Predict(request, 100.0)

print("Time to detect:")
t1 = time.time(); print(t1 - t0)

boxes = np.reshape(res.outputs['detection_boxes'].float_val, [5, 100, 4])
classes = res.outputs['detection_classes'].float_val
scores = res.outputs['detection_scores'].float_val
            
print("Time to unpack response:")
t2 = time.time(); print(t2 - t1)

IMGS ARRAY SHAPE
(5, 1080, 1920, 3)
Time to detect:
1.3263859748840332
Time to unpack response:
0.0021889209747314453
